# LOB Alt

Alternative (simplistic) starter code (generated)

In [1]:
import numpy as np
from dataclasses import dataclass

# ==============================
# Order and Simulator
# ==============================

@dataclass
class Order:
    order_id: int
    side: str          # "buy" or "sell"
    kind: str          # "limit" or "market"
    price: float       # limit price (ignored for pure market orders)
    quantity: int
    time_created: float
    owner: str         # "noise" or "mm"


class LimitOrderBookSimulator:
    def __init__(
        self,
        sim_duration=6.5 * 60 * 60,  # seconds
        tick_size=0.01,
        initial_price=100.0,
        # Poisson arrival rates (orders per second)
        lam_limit_buy=0.5,
        lam_limit_sell=0.5,
        lam_mkt_buy=0.2,
        lam_mkt_sell=0.2,
        lam_cancel=0.3,
        # Market maker parameters
        mm_spread=0.04,     # fixed quoted spread
        mm_skew=0.01,       # inventory skew parameter
        seed=123,
    ):
        self.sim_duration = sim_duration
        self.tick_size = tick_size
        self.initial_price = initial_price

        # Rates
        self.rates = {
            "limit_buy": lam_limit_buy,
            "limit_sell": lam_limit_sell,
            "mkt_buy": lam_mkt_buy,
            "mkt_sell": lam_mkt_sell,
            "cancel": lam_cancel,
        }
        self.total_rate = sum(self.rates.values())

        # RNG
        self.rng = np.random.default_rng(seed)

        # Order book: price-time priority lists
        self.bids = []  # list[Order], sorted by price desc, then time
        self.asks = []  # list[Order], sorted by price asc, then time

        # State
        self.time = 0.0
        self.last_trade_price = initial_price

        # Market maker state
        self.mm_spread = mm_spread
        self.mm_skew = mm_skew
        self.mm_inventory = 0
        self.mm_cash = 0.0
        self.mm_bid_id = None
        self.mm_ask_id = None

        # Order ID counter
        self.next_order_id = 0

        # Metrics
        self.spread_times = []
        self.spreads = []
        self.exec_times = []  # execution times for noise orders
        self.num_trades = 0

    # ------------------------------
    # Helpers
    # ------------------------------
    def _new_order_id(self):
        oid = self.next_order_id
        self.next_order_id += 1
        return oid

    def _round_to_tick(self, x):
        return max(self.tick_size, round(x / self.tick_size) * self.tick_size)

    def _best_bid(self):
        return self.bids[0].price if self.bids else None

    def _best_ask(self):
        return self.asks[0].price if self.asks else None

    def _mid_price(self):
        bb = self._best_bid()
        ba = self._best_ask()
        if bb is not None and ba is not None:
            return 0.5 * (bb + ba)
        return self.last_trade_price

    def _sort_books(self):
        # Price-time priority
        self.bids.sort(key=lambda o: (-o.price, o.time_created, o.order_id))
        self.asks.sort(key=lambda o: (o.price, o.time_created, o.order_id))

    # ------------------------------
    # Market maker quoting
    # ------------------------------
    def _place_mm_limit(self, side, price):
        """Place a single-unit MM limit order; if it does not immediately execute,
        remember its ID as mm_bid_id or mm_ask_id."""
        oid = self._new_order_id()
        order = Order(
            order_id=oid,
            side=side,
            kind="limit",
            price=price,
            quantity=1,
            time_created=self.time,
            owner="mm",
        )
        filled = self._process_limit_order(order)
        if not filled:  # order remains resting
            if side == "buy":
                self.mm_bid_id = oid
            else:
                self.mm_ask_id = oid

    def _update_mm_quotes(self):
        # Ensure MM has a bid and an ask resting (if possible)
        mid = self._mid_price()
        # shift mid based on inventory: if long, shift down; if short, shift up
        mid_adj = mid - self.mm_skew * self.mm_inventory

        bid_price = self._round_to_tick(mid_adj - self.mm_spread / 2.0)
        ask_price = self._round_to_tick(mid_adj + self.mm_spread / 2.0)
        if bid_price <= 0:
            bid_price = self.tick_size

        # Only post new quotes if missing
        if self.mm_bid_id is None:
            self._place_mm_limit("buy", bid_price)
        if self.mm_ask_id is None:
            self._place_mm_limit("sell", ask_price)

    # ------------------------------
    # Order generation
    # ------------------------------
    def _sample_limit_price(self):
        # Sample around last trade price within a small band of ticks
        base = self.last_trade_price
        offset_ticks = self.rng.integers(-5, 6)  # -5..5
        price = base + offset_ticks * self.tick_size
        return self._round_to_tick(max(self.tick_size, price))

    # ------------------------------
    # Matching engine
    # ------------------------------
    def _record_execution_times(self, taker, maker):
        # Taker
        if taker.owner == "noise":
            self.exec_times.append(self.time - taker.time_created)
        # Maker (resting)
        if maker.owner == "noise":
            self.exec_times.append(self.time - maker.time_created)

    def _update_mm_pnl_for_trade(self, buyer, seller, trade_price):
        # Buyer pays cash, gets +1 inventory; Seller gets cash, -1 inventory
        if buyer.owner == "mm":
            self.mm_inventory += 1
            self.mm_cash -= trade_price
        if seller.owner == "mm":
            self.mm_inventory -= 1
            self.mm_cash += trade_price

    def _process_limit_order(self, order):
        """Process incoming limit order, match against book, and
        add any remaining quantity. Returns True if fully filled immediately."""
        side = order.side
        filled_immediately = False

        if side == "buy":
            book = self.asks
            while order.quantity > 0 and book:
                best = book[0]
                if order.price >= best.price:
                    # Trade
                    trade_price = best.price
                    self.num_trades += 1
                    self.last_trade_price = trade_price
                    self._record_execution_times(order, best)
                    self._update_mm_pnl_for_trade(buyer=order, seller=best, trade_price=trade_price)

                    order.quantity -= 1
                    best.quantity -= 1

                    # If resting order emptied, remove it and clear MM IDs if needed
                    if best.quantity == 0:
                        if best.owner == "mm":
                            if best.side == "buy" and self.mm_bid_id == best.order_id:
                                self.mm_bid_id = None
                            if best.side == "sell" and self.mm_ask_id == best.order_id:
                                self.mm_ask_id = None
                        book.pop(0)
                else:
                    break

            if order.quantity > 0:
                # Add remaining to bid book
                self.bids.append(order)
                self._sort_books()
            else:
                filled_immediately = True

        else:  # side == "sell"
            book = self.bids
            while order.quantity > 0 and book:
                best = book[0]
                if order.price <= best.price:
                    trade_price = best.price
                    self.num_trades += 1
                    self.last_trade_price = trade_price
                    self._record_execution_times(order, best)
                    self._update_mm_pnl_for_trade(buyer=best, seller=order, trade_price=trade_price)

                    order.quantity -= 1
                    best.quantity -= 1

                    if best.quantity == 0:
                        if best.owner == "mm":
                            if best.side == "buy" and self.mm_bid_id == best.order_id:
                                self.mm_bid_id = None
                            if best.side == "sell" and self.mm_ask_id == best.order_id:
                                self.mm_ask_id = None
                        book.pop(0)
                else:
                    break

            if order.quantity > 0:
                # Add remaining to ask book
                self.asks.append(order)
                self._sort_books()
            else:
                filled_immediately = True

        return filled_immediately

    def _process_market_order(self, side):
        order = Order(
            order_id=self._new_order_id(),
            side=side,
            kind="market",
            price=0.0,
            quantity=1,
            time_created=self.time,
            owner="noise",
        )

        if side == "buy":
            book = self.asks
            if book:
                best = book[0]
                trade_price = best.price
                self.num_trades += 1
                self.last_trade_price = trade_price
                self._record_execution_times(order, best)
                self._update_mm_pnl_for_trade(buyer=order, seller=best, trade_price=trade_price)

                order.quantity -= 1
                best.quantity -= 1
                if best.quantity == 0:
                    if best.owner == "mm" and self.mm_ask_id == best.order_id:
                        self.mm_ask_id = None
                    book.pop(0)
        else:  # sell
            book = self.bids
            if book:
                best = book[0]
                trade_price = best.price
                self.num_trades += 1
                self.last_trade_price = trade_price
                self._record_execution_times(order, best)
                self._update_mm_pnl_for_trade(buyer=best, seller=order, trade_price=trade_price)

                order.quantity -= 1
                best.quantity -= 1
                if best.quantity == 0:
                    if best.owner == "mm" and self.mm_bid_id == best.order_id:
                        self.mm_bid_id = None
                    book.pop(0)

    def _process_cancellation(self):
        # Randomly cancel one resting order (if any)
        total_resting = len(self.bids) + len(self.asks)
        if total_resting == 0:
            return

        idx = self.rng.integers(0, total_resting)
        if idx < len(self.bids):
            order = self.bids.pop(idx)
        else:
            order = self.asks.pop(idx - len(self.bids))

        # If we canceled MM order, clear its ID
        if order.owner == "mm":
            if order.side == "buy" and self.mm_bid_id == order.order_id:
                self.mm_bid_id = None
            if order.side == "sell" and self.mm_ask_id == order.order_id:
                self.mm_ask_id = None

    # ------------------------------
    # Main simulation loop
    # ------------------------------
    def _record_spread(self):
        bb = self._best_bid()
        ba = self._best_ask()
        if bb is not None and ba is not None:
            self.spread_times.append(self.time)
            self.spreads.append(ba - bb)

    def run(self):
        # Bootstrap MM quotes so there is some initial liquidity
        self._update_mm_quotes()
        self._record_spread()

        if self.total_rate <= 0:
            raise ValueError("Total arrival rate must be positive.")

        event_names = list(self.rates.keys())
        event_rates = np.array([self.rates[name] for name in event_names], dtype=float)
        event_probs = event_rates / self.total_rate

        while self.time < self.sim_duration:
            # Poisson next-event time increment
            dt = self.rng.exponential(1.0 / self.total_rate)
            self.time += dt

            # Sample which event type occurred
            event = self.rng.choice(event_names, p=event_probs)

            if event == "limit_buy":
                price = self._sample_limit_price()
                order = Order(
                    order_id=self._new_order_id(),
                    side="buy",
                    kind="limit",
                    price=price,
                    quantity=1,
                    time_created=self.time,
                    owner="noise",
                )
                self._process_limit_order(order)

            elif event == "limit_sell":
                price = self._sample_limit_price()
                order = Order(
                    order_id=self._new_order_id(),
                    side="sell",
                    kind="limit",
                    price=price,
                    quantity=1,
                    time_created=self.time,
                    owner="noise",
                )
                self._process_limit_order(order)

            elif event == "mkt_buy":
                self._process_market_order("buy")

            elif event == "mkt_sell":
                self._process_market_order("sell")

            elif event == "cancel":
                self._process_cancellation()

            # Update MM quotes (simple strategy)
            self._update_mm_quotes()

            # Record spread
            self._record_spread()

        # Final MM PnL (mark inventory to last trade price)
        final_mark = self.last_trade_price
        mm_pnl = self.mm_cash + self.mm_inventory * final_mark

        # Basic summaries for analysis
        avg_spread = np.mean(self.spreads) if self.spreads else np.nan
        median_exec_time = np.median(self.exec_times) if self.exec_times else np.nan

        return {
            "avg_spread": avg_spread,
            "median_exec_time": median_exec_time,
            "mm_pnl": mm_pnl,
            "num_trades": self.num_trades,
            "exec_times": self.exec_times,
            "spreads": self.spreads,
            "spread_times": self.spread_times,
        }


# ==============================
# Multiple replications example
# ==============================

if __name__ == "__main__":
    NUM_REPS = 3

    results = []
    for r in range(NUM_REPS):
        sim = LimitOrderBookSimulator(
            sim_duration=60 * 60,  # 1 hour for faster tests
            lam_limit_buy=0.6,
            lam_limit_sell=0.6,
            lam_mkt_buy=0.2,
            lam_mkt_sell=0.2,
            lam_cancel=0.3,
            mm_spread=0.04,
            mm_skew=0.01,
            seed=123 + r,
        )
        out = sim.run()
        results.append(out)
        print(f"Replication {r}: "
              f"avg_spread={out['avg_spread']:.4f}, "
              f"median_exec_time={out['median_exec_time']:.2f}, "
              f"mm_pnl={out['mm_pnl']:.2f}, "
              f"num_trades={out['num_trades']}")

    # At this point `results` holds all per-replication metrics for further analysis


Replication 0: avg_spread=0.0557, median_exec_time=0.00, mm_pnl=-22.90, num_trades=2869
Replication 1: avg_spread=0.0597, median_exec_time=0.00, mm_pnl=-18.84, num_trades=2904
Replication 2: avg_spread=0.0602, median_exec_time=0.00, mm_pnl=-10.10, num_trades=2897
